In [ ]:
"""
Moduł zawiera funkcje przeznaczone do pracy z bazą danych SQLite.
"""

import sqlite3
import pandas as pd
import json
import sqlalchemy as sa
import shutil

def table_to_json(db_path, table):
    """
    Przepisuje zawartość tabeli z bazy danych SQLite do pliku JSON.
    """
    engine = sa.create_engine(f"sqlite:///{db_path}")
    
    query = f"SELECT * FROM {table}"
    df = pd.read_sql(query, engine)
    
    json_file = table + ".json"
    df.to_json(json_file, orient='records', force_ascii=False, indent=4)
    
    print(f"Dane wyeksportowano do pliku '{json_file}'.")
    return df.head()

def json_to_table(db_path, table, json_file):
    """
    Prepisuje zawartość pliku JSON do tabeli w bazie SQLite.
    Jeśli tabela o podanej nazwie już istnieje, zostanie zamieniona.
    """
    with open(json_file, encoding='utf-8') as f:
        data = json.load(f)
    
    df = pd.DataFrame(data)
    
    engine = sa.create_engine(f"sqlite:///{db_path}")
    
    df.to_sql(table, con=engine, if_exists='replace', index=False)
    
    print(f"Dane zapisano do tabeli '{table}' w bazie danych.")

def create_backup(db_file, db_backup_file):
    """
    Tworzy kopię bazy danych.
    """
    shutil.copy2(db_file, db_backup_file)
    print(f"Kopia bazy '{db_file}' została zapisana jako '{db_backup_file}'.")

def clear_db(db_file):
    """
    Czyści zawartość bazy danych.
    """
    engine = sa.create_engine(f"sqlite:///{db_file}")
    meta = sa.MetaData()
    meta.reflect(bind = engine)

    with engine.begin() as conn:
        for table in reversed(meta.sorted_tables):
            conn.execute(sa.text(f"DROP TABLE IF EXISTS {table.name} CASCADE;"))
    print("Baza danych została wyczyszczona.")